We'll be building a simple data feature that takes a movie id, and provides the box office sum and unemployment rate when that movie was released.

We will use two APIs:


1.   An API that provides movie related info (OMDb API)
2.   An API that provides the unemployment rate (FRED API)

Steps:


*   Sign up for OMDb API key
*   Sign up for FRED API key
*   Add APIs to Colab secrets + test APIs
*   Write function to get movie details (including title) by movie ID
*   Write function to get unemployment data based on the movie release day










API Rate Limits:


*   The rate limit for OMDb is 1,000 per _day_.
*   The rate limit for FRED is 120 per _minute_.



In [20]:
#set up API keys
from google.colab import userdata

OMDB_API_KEYS = userdata.get('OMDB_API_KEY')
FRED_API_KEYS = userdata.get('FRED_API_KEY')

if OMDB_API_KEYS:
  print('OMDB key retrieved successfully.')
else:
  print('Could not retrieve OMDB key.')

if FRED_API_KEYS:
  print('FRED key retrieved successfully.')
else:
  print('Could not find FRED :(')

OMDB key retrieved successfully.
FRED key retrieved successfully.


In [21]:
import requests

#Function to get movie information (title, year, and plot) using movie name (OMDb API)
#Note: While this is not caps-sensitive, please make sure the movie title is correctly spelled.
def get_movie_by_name(imdb_name):
    url = f"https://www.omdbapi.com/?t={imdb_name}&apikey={OMDB_API_KEYS}"
    response = requests.get(url)

    if response.status_code == 200:
        movie_data = response.json()
        return movie_data
    else:
        print(f"Error: {response.status_code}")
        return None

In [22]:
#Test with Inception
movie = get_movie_by_name("Inception")

if movie:
    print(f"Title: {movie['Title']}")
    print(f"Year: {movie['Year']}")
    print(f"Date Released: {movie['Released']}")
    print(f"Plot: {movie['Plot']}")

Title: Inception
Year: 2010
Date Released: 16 Jul 2010
Plot: A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O., but his tragic past may doom the project and his team to disaster.


In [23]:
#Function to convert the date format into one FRED likes
from datetime import datetime

def convert_date_format(date_str):
    # Define the input format
    input_format = "%d %b %Y"
    # Parse the input date string into a datetime object
    date_obj = datetime.strptime(date_str, input_format)
    # Convert the datetime object to the desired output format
    output_format = "%Y-%m-%d"  # Uses '-' to remove leading zeros
    return date_obj.strftime(output_format)

#Example usage:
date_input = "16 Jul 2010"
converted_date = convert_date_format(date_input)
print(f"Converted date: {converted_date}")


Converted date: 2010-07-16


In [24]:
#Get unemployment data from FRED
def get_unemployment_data(date):
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id=UNRATE&api_key={FRED_API_KEYS}&file_type=json&observation_start={date}&observation_end={date}"
    response = requests.get(url)

    if response.status_code == 200:
        raw_data = response.json()
        unrate_data = raw_data['observations'][0]['value'] + "%"
        return unrate_data
    else:
        print(f"Error: {response.status_code}")
        return None

#Test the function with Inception's date
get_unemployment_data("2010-07-16")


'9.4%'

In [25]:
#Combine movie + unemployment data

def movie_ployment(title):
  movie = get_movie_by_name(title)
  raw_date = movie['Released']
  date = convert_date_format(raw_date)
  unrate = get_unemployment_data(date)
  revenue = movie['BoxOffice']
  return f"{title} received {revenue} at the box office. The unemployment rate at the time was {unrate}."

In [26]:
#Test final output, using Inception

movie_ployment("Inception")

'Inception received $292,587,330 at the box office. The unemployment rate at the time was 9.4%.'

In [27]:
title = input("Enter a movie title: ")
movie_ployment(title)

Enter a movie title: Inside Out 2


'Inside Out 2 received $646,311,114 at the box office. The unemployment rate at the time was 4.1%.'